In [280]:
import requests
import urllib.parse
from bs4 import BeautifulSoup



def tahlilgaran_translate(word):
    url = f"http://tahlilgaran.org/TDictionary/WebApp/?q={word}"
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        translation_div = soup.find('div', class_='p-fa')
        if translation_div:
            return translation_div.get_text(strip=True)
    return ''


class Token:
    def __init__(self, tkk):
        self.tkk = tkk

    def calculate_token(self, text):
        if self.tkk == "":
            self.tkk = "440498.1287591069"
        [first_seed, second_seed] = self.tkk.split(".")

        try:
            d = bytearray(text.encode('UTF-8'))
        except UnicodeDecodeError:
            d = bytearray(text)

        a = int(first_seed)
        for value in d:
            a += value
            a = self._work_token(a, "+-a^+6")
        a = self._work_token(a, "+-3^+b+-f")
        a ^= int(second_seed)
        if 0 > a:
            a = (a & 2147483647) + 2147483648
        a %= 1E6
        a = int(a)
        return str(a) + "." + str(a ^ int(first_seed))

    @staticmethod
    def _rshift(val, n):
        return val >> n if val >= 0 else (val + 0x100000000) >> n

    def _work_token(self, a, seed):
        for i in range(0, len(seed) - 2, 3):
            char = seed[i + 2]
            d = ord(char[0]) - 87 if char >= "a" else int(char)
            d = self._rshift(a, d) if seed[i + 1] == "+" else a << d
            a = a + d & 4294967295 if seed[i] == "+" else a ^ d
        return a


class GoogleTranslate:
    def __init__(self, target_language, proxy=None):
        self.http_host = 'translate.googleapis.com'
        self.proxy = proxy
        self.target_language = target_language
        self.result = ''

    def get_url(self, tl, qry, tk):
        url = f'https://{self.http_host}/translate_a/single?client=gtx&sl=en&tl={tl}&hl=en&dt=at&dt=bd&dt=ex&' \
              f'dt=ld&dt=md&dt=qca&dt=rw&dt=rm&dt=sos&dt=ss&dt=t&ssel=0&tsel=0&kc=1&tk={tk}&q={qry}'
        return url

    def get_resp(self, url):
        base_headers = {'User-Agent': 'Mozilla/5.0'}
        session = requests.Session()
        session.headers = base_headers
        resp = session.get(url, timeout=5)
        return resp.json()
    def translate(self, word, tkk=''):
        tk = Token(tkk).calculate_token(word)
        parse_query = urllib.parse.quote_plus(word)
        url = self.get_url(self.target_language, parse_query, tk)

        response = self.get_resp(url)

        
        return format_result(response) + '\n' + tahlilgaran_translate(word)

def format_result(result):
    output = []
    output = [result[0][0][0], result[0][0][1], "---"]
    if result[1]:
        for wt in result[1]:
            output.append(f"{wt[3]}")  
            output.append(f"{wt[0]}") 
            output.append(', '.join(wt[1]))  
            output.append("")  

            if wt[2]:  
                for w in wt[2]:
                    line = f"{w[0]}: {', '.join(w[1])}" 
                    if len(w) == 4:  
                        line += f" {int(w[3]*100)}"
                    output.append(line)
            output.append("_"*23)  
    return '\n'.join(output)

translator = GoogleTranslate('fa')  
result = translator.translate('rounded scripts')


In [ ]:
import os
import re
import pandas as pd
from glob import glob
from gtts import gTTS
import time

base_dir = '/home/ri/Desktop/TOEFL/TPO-NEO/Finished'

for TPO in os.listdir(base_dir):
    tpo_dir = os.path.join(base_dir, TPO)
    tpo = os.path.basename(tpo_dir)
    audio_dir = os.path.join('/home/ri/.local/share/Anki2/fc1/collection.media/', tpo) 
    if not os.path.exists(audio_dir):
        os.makedirs(audio_dir)

        csv_files = glob(tpo_dir + '/*.csv')
        assert len(csv_files) == 1, f"Expected one CSV file in {tpo_dir}, but found {len(csv_files)}."
        csv_file = csv_files[0]
        
        df = pd.read_csv(csv_file, header=None)
        words = df.iloc[:, 3].tolist()  
        
        words_translate = {}
        words_audio = {}

        df['Translation'] = ""
        df['Audio_File'] = ""

        for idx, word in enumerate(words):
            sanitized_word = re.sub(r'[^\w\s-]', '', word).strip().replace(' ', '_')
            file_path = os.path.join(audio_dir, f"{sanitized_word}.mp3")
            
            try:
                tts = gTTS(text=word, lang='en-us', tld='us', slow=False)
                tts.save(file_path)
                words_audio[word] = file_path

                df.at[idx, 'Audio_File'] = f'[sound:{tpo}/{sanitized_word}]'

                attempts = 0
                max_attempts = 5  
                while attempts < max_attempts:
                    try:
                        translator = GoogleTranslate('fa') 
                        translated_text = translator.translate(word)
                        words_translate[word] = translated_text
                        
                        df.at[idx, 'Translation'] = translated_text
                        break  
                    except Exception as e:
                        attempts += 1
                        print(f"Error translating {word}, attempt {attempts}. Error: {e}")
                        if attempts < max_attempts:
                            time.sleep(3)  
                        else:
                            print(f"Failed to translate {word} after {max_attempts} attempts.")
                            df.at[idx, 'Translation'] = ''
            except Exception as e:
                print(f"Error generating pronunciation for {word}: {e}")
                df.at[idx, 'Audio_File'] = ''

        output_csv_file = os.path.join(tpo_dir, f"updated_{os.path.basename(csv_file)}")
        df.to_csv(output_csv_file, index=False, header=None)
        print(f"Saved updated CSV: {output_csv_file}")
        time.sleep(5)


In [ ]:
from gtts import gTTS
from IPython.display import Audio

tts = gTTS(text='water', lang='en-us', tld='us', slow=False)
tts.save("water.mp3")
Audio("water.mp3", autoplay=True)


In [ ]:
tts = gTTS(text='water', lang='en', slow=False)
tts.save("water.mp3")
Audio("water.mp3", autoplay=True)